## Queries - Gen3

In this notebook, we show several ways to query the Gen3 data\
Craig Lage - 25-Jul-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
import pandas as pd
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.display as afwDisplay
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.daf.butler import Butler

In [ ]:
butler = butlerUtils.makeDefaultButler("LATISS")
instrument = 'LATISS'

In [ ]:
butler = Butler('/repo/embargo_old', collections="LATISS/raw/all")

In [ ]:
dayObs = 20250318

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter, record.timespan.begin.isot)#, \


In [ ]:
record.timespan.begin.isot

## Define a simple ISR
### Just overscan subtraction and bias subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR and look at the result

In [ ]:
expId = 2025022500608
#expId = 2025022500202
#expId = 2024070800403
instrument='LATISS'
exp = butler.get('raw', detector=0, exposure=expId, instrument=instrument)
isrResult = isrTask.run(exp) # This runs the ISR

In [ ]:
%matplotlib inline
x = plot(isrResult.exposure, stretch='linear')
#plt.savefig(f"/home/c/cslage/u/AuxTel/stuttered/AuxTel_Stuttered_{expId}.png")

In [ ]:
y = 2000
x1 = 1700
x2 = 1800
xs = list(range(x1, x2))
fluxes = isrResult.exposure.image.array[y, x1:x2]
plt.plot(xs, fluxes)

# Trying DeepSeek's version

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def gaussian(x, bg, A, mu, sigma):
    """
    Gaussian function
    """
    return bg + A * np.exp(-(x - mu)**2 / (2 * sigma**2))

def fit_gaussian(position_data, flux_data):
    """
    Fits a Gaussian curve to flux vs position data
    
    Args:
        position_data: 1D array of position values
        flux_data: 1D array of corresponding flux values
    
    Returns:
        Tuple of (A, mu, sigma) - optimal parameters for the Gaussian fit
    """
    # Calculate initial parameter guesses
    bg_guess = 0
    A_guess = np.max(flux_data)
    weighted_sum = np.sum(flux_data * position_data)
    total_flux = np.sum(flux_data)
    
    if total_flux == 0:
        raise ValueError("Total flux cannot be zero for fitting")
    
    mu_guess = weighted_sum / total_flux
    sigma_guess = np.sqrt(np.sum(flux_data * (position_data - mu_guess)**2) / total_flux)
    
    # Perform the curve fit
    params, _ = curve_fit(gaussian, 
                         position_data, 
                         flux_data, 
                         p0=[bg_guess, A_guess, mu_guess, sigma_guess])
    
    return tuple(params)

In [ ]:
bg, A, mu, sigma = fit_gaussian(xs, fluxes)
print(bg, A, mu, sigma)

In [ ]:
result.x

In [ ]:
y_fit = gaussian(xs, bg, A, mu, sigma)
plt.plot(xs, fluxes)
plt.plot(xs, y_fit, marker='x')
plt.title(f"LATISS Spectra {expId}, Y={y}")
fwhm = sigma / 10.0 * 2.355
plt.text(mu+20, 20000, f"FWHM = {fwhm:.2f} arcsec")
plt.savefig(f'/home/c/cslage/u/AuxTel/fwhm/Spectrum_FWHM_{expId}.png')

In [ ]:
xs[0:3]

In [ ]:
from scipy.optimize import minimize
def FOM(params, args):
    fom = 0.0
    [xs, fluxes] = args
    [bg, mag, mean, sigma] = params
    for i in range(len(xs)):
        model = bg + mag * np.exp(-(xs[i] - mean)**2 / (2 * sigma**2))
        fom += np.square(model - fluxes[i])
    return fom

args = [xs, fluxes]
x0 = [1, 4000.0, 1750, 10]
bounds=[(0, 5000), (1000, 200000), (1700, 1800), (2, 20)]
result = minimize(FOM, x0, args=args, bounds=bounds, method='Powell')
result

In [ ]:
[bg, mag, mean, sigma] = result.x
fwhm = sigma / 10.0 * 2.355
models = []
for i in range(len(xs)):
    model = bg + mag * np.exp(-(xs[i] - mean)**2 / (2 * sigma**2))
    models.append(model)
plt.plot(xs, fluxes)
plt.plot(xs, models, marker='x')
plt.text(1600, max(fluxes) / 2.0, f"FWHM = {fwhm:.2f} arcsec")


In [ ]:
sum = np.zeros(len(xs))
for i in range(-8, 3,1):
    for j in range(len(xs)):
        index = j + i * 10
        if index < 0 or index > x2-x1-1:
            continue
        sum[index] += fluxes[j]
plt.plot(xs, sum)

In [ ]:
expId = 2024022200254
exp = butler.get('raw', detector=0, exposure=expId)


In [ ]:
dataId = dict(day_obs=20240222, detector=0, seq_num=254)

dataType = 'quickLookExp'
exp = butler.get(dataType, dataId)

In [ ]:
expId = 2024101600431
exp = butler.get('raw', detector=0, exposure=expId)


In [ ]:
for record in records:
    print(record.id)

In [ ]:
import lsst.summit.utils.butlerUtils as butlerUtils
butler = butlerUtils.makeDefaultButler("LATISS")

In [ ]:
dayObs = 20241217
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    mData = butler.get('raw.metadata', detector=0, exposure=id)
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)#, \
    #      record.target_name, mData['SEQFILE'])

In [ ]:
dayObs = 20230510
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    expId = record.id
    print(expId, int(expId - dayObs * 1E5))
    break


In [ ]:
dayObs = 20220503
exposureList = []
for record in old_butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    mData = old_butler.get('raw.metadata', detector=0, exposure=id)
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)#, \
    #      record.target_name, mData['SEQFILE'])

In [ ]:
record.azimuth

In [ ]:
dataId={'instrument':'LATISS', 'detector':0, 'exposure':2022092700233}

In [ ]:
test1 = butler.getURI('raw', dataId={'instrument':'LATISS', 'detector':0, 'exposure':2022092700233})

In [ ]:
test.relativeToPathRoot

In [ ]:
test.geturl()

In [ ]:
test.path

In [ ]:
data = butler.get('raw', dataId={'instrument':'LATISS', 'detector':0, 'exposure':2023050800002})

In [ ]:
plt.imshow(test.image.array, interpolation='Nearest')

In [ ]:
test.readFitsWithOptions?

In [ ]:
for i in test:
    print(i)

In [ ]:
FWHM_list

In [ ]:
CD1_1 0.0552614153694827
CD1_2 0.089281442480291
CD2_1 -0.089281442480291
CD2_2 0.0552614153694827

In [ ]:
expId = 2023031400200
mData = butler.get('raw.metadata', detector=0, exposure=expId)
print(mData['EXPTIME'])


In [ ]:
expId = 2023051000565
mData = butler.get('raw.metadata', detector=0, exposure=expId)
print(mData['SEQFILE'])

In [ ]:
expId = 2025012100004
mData = butler.get('raw.metadata', detector=0, exposure=expId)
for key in mData.keys():
    print(key, mData[key])


In [ ]:
ras = []
decs = []
for expId in range(2025011500574, 2025011500580):
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    for key in mData.keys():
        if key in ['RASTART', 'DECSTART']:
            #print(expId, key, mData[key])
            ras.append(float(mData['RASTART']))
            decs.append(float(mData['DECSTART']))

    dRA = (ras[-1] - ras[0]) * 3600.0
    dDec = (decs[-1] - decs[0]) * 3600.0
    print(expId, dRA, dDec)
    
    

In [ ]:
expId=2024082700510
mData = butler.get('raw.metadata', detector=0, exposure=expId)
for key in mData.keys():
    print(key, mData[key])
   

In [ ]:
for key in mData.keys():
    if key in ['AP0_RC', 'AP0_GAIN','AP1_RC', 'AP1_GAIN']:
        print(key, mData[key])

In [ ]:
int(mData['DETSIZE'].split(',')[0].split(':')[1])

In [ ]:
expId = 2022102600437
exp = butler.get('raw', detector=0, exposure=expId)

In [ ]:
# Look at the data with matplotlib
# The raw data doesn't look very good, because of the large pedestal of about 15,000 ADU
from matplotlib.colors import LogNorm
# Now let's look at ithem
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = exp.image.array
img = plt.imshow(arr, norm=LogNorm(vmin=12000, vmax=20000), interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)
#plt.savefig(REPO_DIR+"/plots/NGC4755_17Feb21.pdf")

In [ ]:
plt.plot(arr[1000,0:509])

In [ ]:
np.median(arr)

In [ ]:
dayObs = 20210402
expId = 2021040200034
butler = Butler('/repo/main', collections="LSSTComCam/raw/all")
mData = butler.get('raw.metadata', detector=4, exposure=expId)

In [ ]:
mData = butler.get('raw.metadata', detector=4, exposure=expId)
for key in mData.keys():
    print(key, mData[key])


In [ ]:
expId = 3021120600576
exp = butler.get('raw', detector=55, exposure=expId)

In [ ]:
# Look at the data with matplotlib
# The raw data doesn't look very good, because of the large pedestal of about 15,000 ADU
from matplotlib.colors import LogNorm
# Now let's look at ithem
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = exp.image.array
img = plt.imshow(arr, norm=LogNorm(vmin=10, vmax=100000), interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)
#plt.savefig(REPO_DIR+"/plots/NGC4755_17Feb21.pdf")

In [ ]:
arr.std()

In [ ]:
butler = Butler('/repo/main', collections=['LATISS/raw/all','LATISS/calib','u/cslage/calib/latiss/calib.2021021'])

In [ ]:
expId = 2021021700077
exp = butler.get('raw', detector=0, exposure=expId)

In [ ]:
expId = 2021021700090
bias = butler.get('bias', detector=0, exposure=expId)

In [ ]:
butler = Butler("/repo/main", collections=["LSSTCam/raw/all","LSSTCam/calib","u/cslage/calib/13144/calib.20220103",\
                                           "u/cslage/tests/linearizer_dm33297_21jan22"])

In [ ]:
expId = 3021120600576

In [ ]:
bias = butler.get('bias', detector=55, exposure=expId)

In [ ]:
defect = butler.get('defects', detector=55, exposure=expId)

In [ ]:
ptc = butler.get('ptc', detector=55, exposure=expId)

In [ ]:
lin = butler.get('linearizer', detector=55, exposure=expId)

In [ ]:
butler = Butler('/repo/oga', collections=["LATISS/raw/all", "LATISS/calib"])
mBias = butler.get('bias', detector=0, exposure=2022092900947)

In [ ]:
type(mBias)

In [ ]:
# Gen3 butler
for month in [1,2,3]:
    for day in range(31):
        dayObs = int(f"2023{month:02}{day:02}")
        try:
            exposureList = []
            for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
                exposureList.append([record.id, record])
            exposureList.sort(key=lambda x: x[0])
            for [id,record] in exposureList:
                if record.observation_type == 'dark' and record.exposure_time > 60.0:
                    print(dayObs, id, record.exposure_time)
        except:
            continue

In [ ]:
record.observation_type, record.exposure_time

In [ ]:
from lsst.daf.butler import Butler
embargoButler = Butler('/repo/embargo', collections="LATISS/raw/all")
mainButler = Butler('/repo/main', collections="LATISS/raw/all")
dates = []
RCs = []
for year in [2022, 2023]:
    for month in range(1,13):
        if year == 2021:
            butler = mainButler
        if year == 2022:
            if month < 9:
                butler = mainButler
            else:
                butler = embargoButler
        if year == 2023:
            butler = embargoButler
        for day in range(1,30):
            dayObs = int(f"{year}{month:02}{day:02}")
            expId = dayObs * 100000 + 1
            try:
                mData = butler.get('raw.metadata', detector=0, exposure=expId)
                print(f"{year}-{month:02}-{day:02} AP0_RC = {mData['AP0_RC']}")
                
                RCs.append(mData['AP0_RC'])
                date = pd.Timestamp(f"{year}-{month:02}-{day:02}")
                dates.append(date)
                continue
            except:
                continue

In [ ]:
print(len(dates), len(RCs))

In [ ]:
df = pd.DataFrame(RCs, dates, columns =['RC'])

In [ ]:
df.head(3)

In [ ]:
df.plot()

In [ ]:
df

In [ ]:
plt.plot(dates, RCs)

In [ ]:
dayObss

In [ ]:
date = Time(f"{year}-{month:02}-{day:02}T00:00:00")

In [ ]:
date.isot

In [ ]:
test = [3,7]

In [ ]:
if test:
    print("True")

In [ ]:
from lsst.daf.butler import Butler
butler = Butler('/repo/embargo', collections=["LSSTComCamSim/raw/all", "LSSTComCamSim/defaults", "LSSTComCamSim/quickLook/24"])

In [ ]:
butler.registry.queryDatasets(
        "raw",
        collections="LSSTComCamSim/raw/all",
        where="exposure.science_program='ops-rehearsal-3' and instrument='LSSTComCamSim' and exposure.day_obs=20240403 ",
).count()

In [ ]:
dayObs = 20240403

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", \
    where="exposure.science_program='ops-rehearsal-3' and instrument='LSSTComCamSim' and exposure.day_obs=20240403"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


In [ ]:
expId = 7024040300049
butler.get('raw', detector=4, exposure=expId, instrument='LSSTComCamSim')

In [ ]:
expId = 7024040300049
mData = butler.get('raw.metadata', detector=4, exposure=expId)
for key in mData.keys():
    print(key, mData[key])


In [ ]:
expId = 2025022500608
#expId = 2025022500202
#expId = 2024070800403
instrument='LATISS'
raw = butler.get('raw', detector=0, exposure=expId, instrument=instrument)

uri = butler.getURI('raw', dataId={'instrument': 'LATISS', 'detector':0, 'exposure':expId})
print(uri)